In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import os.path as osp

import numpy as np

In [2]:
def create_label_map(labels):
    label_set = set()
    for lt in labels:
        label_set.add(lt)

    label_map = {}
    count = 0
    for l in label_set:
        label_map[l] = count
        count += 1
        
    return label_map

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding="same")
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding="same")
        #self.norm2a = nn.BatchNorm2d(32)
        self.norm2b = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding="same")
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding="same")
        #self.norm4a = nn.BatchNorm2d(64)
        self.norm4b = nn.BatchNorm2d(64)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding="same")
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding="same")
        #self.conv7 = nn.Conv2d(128, 128, kernel_size=3, padding="same")
        #self.norm7a = nn.BatchNorm2d(128)
        self.norm7b = nn.BatchNorm2d(128)
        
        self.conv8 = nn.Conv2d(128, 256, kernel_size=3, padding="same")
        self.conv9 = nn.Conv2d(256, 256, kernel_size=3, padding="same")
        #self.conv10 = nn.Conv2d(256, 256, kernel_size=3, padding="same")
        #self.norm10a = nn.BatchNorm2d(256)
        self.norm10b = nn.BatchNorm2d(256)
        
        self.conv11 = nn.Conv2d(256, 512, kernel_size=3, padding="same")
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding="same")
        #self.conv13 = nn.Conv2d(512, 512, kernel_size=3, padding="same")
        #self.norm13a = nn.BatchNorm2d(512)
        self.norm13b = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(8192, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        #x = self.norm2a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm2b(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        #x = self.norm4a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm4b(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        #x = F.relu(self.conv7(x))
        #x = self.norm7a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm7b(x)
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        #x = F.relu(self.conv10(x))
        #x = self.norm10a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm10b(x)
        
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        #x = F.relu(self.conv13(x))
        #x = self.norm13a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm13b(x)
        
        # print(x.shape)
        x = x.view(-1, 8192)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        
        x = self.fc3(x)
        return x

In [4]:
def train(
        epoch,
        model,
        loss_func,
        train_loader,
        optimizer,
        log_interval,
        save_path
):  
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):

        optimizer.zero_grad()
        output = model(data)

        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:

            trained_count = batch_idx * len(data)
            total_count = len(train_loader.dataset)
            batch_percent = int(100 * batch_idx / len(train_loader))
            loss_val = loss.item() / len(data)
            print(
                f'Train Epoch: {epoch} ' + 
                f'[{trained_count}/{total_count} ({batch_percent}%)]' + 
                f'\tLoss: {loss_val:.6f}'
            )

    torch.save(model.state_dict(), osp.join(save_path, f"model_{epoch}.pt"))
    torch.save(optimizer.state_dict(), osp.join(save_path, f"opt_{epoch}.pt"))
    
    return loss.item() / len(data)


def test(
    model,
    loss_func,
    test_loader,
    test_type="Validation"
):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_func(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            
        test_loss /= len(test_loader.dataset)
        
        print(
            f'{test_type} Average loss: {test_loss:.4f}, ' +
            f'Accuracy: {correct}/{len(test_loader.dataset)} ' + 
            f'({100.*correct/len(test_loader.dataset):.0f}%)'
        )
    
    return test_loss

In [18]:
n_epochs = 10
batch_size_train = 128
batch_size_valid = 128
batch_size_test = 128
learning_rate = 0.0001
log_interval = 20
save_path = "/home/ubuntu/data/yg_ar/cnn_torch/res0002"

if not osp.exists(save_path):
    os.makedirs(save_path)

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [6]:
from nebula.data.yg_ar.setup_data_image_hard import read_data
from nebula.common import to_scale_one, write_pickle, read_pickle

In [7]:
df_path = "/home/ubuntu/data/yg_ar/image_hard_df.pkl"
random_seed = 1
df, train_df, test_df, valid_df = read_data(df_path, random_seed)

In [8]:
label_map_a = create_label_map(df["label_a"])
label_map_at = create_label_map(df["label_at"])

In [9]:
train_x = train_df["image"].apply(lambda x: np.array([x.astype(np.float32)/225.0])).to_list()
train_y_a = train_df["label_a"].map(label_map_a).to_list()
train_y_at = train_df["label_at"].map(label_map_at).to_list()

In [10]:
valid_x = valid_df["image"].apply(lambda x:  np.array([x.astype(np.float32)/225.0])).to_list()
valid_y_a = valid_df["label_a"].map(label_map_a).to_list()
valid_y_at = valid_df["label_at"].map(label_map_at).to_list()

In [11]:
test_x = test_df["image"].apply(lambda x:  np.array([x.astype(np.float32)/225.0])).to_list()
test_y_a = test_df["label_a"].map(label_map_a).to_list()
test_y_at = test_df["label_at"].map(label_map_at).to_list()

In [19]:
train_loader_a = torch.utils.data.DataLoader(tuple(zip(train_x, train_y_a)), batch_size=batch_size_train, shuffle=True)
train_loader_at = torch.utils.data.DataLoader(tuple(zip(train_x, train_y_at)), batch_size=batch_size_train, shuffle=True)

valid_loader_a = torch.utils.data.DataLoader(tuple(zip(valid_x, valid_y_a)), batch_size=batch_size_valid, shuffle=True)
valid_loader_at = torch.utils.data.DataLoader(tuple(zip(valid_x, valid_y_at)), batch_size=batch_size_valid, shuffle=True)

test_loader_a = torch.utils.data.DataLoader(tuple(zip(test_x, test_y_a)), batch_size=batch_size_test, shuffle=True)
test_loader_at = torch.utils.data.DataLoader(tuple(zip(test_x, test_y_at)), batch_size=batch_size_test, shuffle=True)

model = Net()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss(size_average=False)

train_res = []
test_res = []

test(
    model=model,
    loss_func=loss_func,
    test_loader=valid_loader_at
)

for epoch in range(1, n_epochs + 1):
    train_loss = train(
        epoch=epoch,
        model=model,
        loss_func=loss_func,
        train_loader=train_loader_at,
        optimizer=optimizer,
        log_interval=log_interval,
        save_path=save_path
    )
    train_res.append((epoch, train_loss))

    test_loss = test(
        model=model,
        loss_func=loss_func,
        test_loader=valid_loader_at
    )
    test_res.append((epoch, test_loss))

Validation Average loss: 2.3029, Accuracy: 176/1760 (10%)
Train Epoch: 1 [0/16080 (0%)]	Loss: 2.304047
Train Epoch: 1 [1280/16080 (7%)]	Loss: 2.348608
Train Epoch: 1 [2560/16080 (15%)]	Loss: 2.210284
Train Epoch: 1 [3840/16080 (23%)]	Loss: 2.199447
Train Epoch: 1 [5120/16080 (31%)]	Loss: 2.058052
Train Epoch: 1 [6400/16080 (39%)]	Loss: 1.894537
Train Epoch: 1 [7680/16080 (47%)]	Loss: 1.807622
Train Epoch: 1 [8960/16080 (55%)]	Loss: 1.550088
Train Epoch: 1 [10240/16080 (63%)]	Loss: 1.482506
Train Epoch: 1 [11520/16080 (71%)]	Loss: 1.407411
Train Epoch: 1 [12800/16080 (79%)]	Loss: 1.083449
Train Epoch: 1 [14080/16080 (87%)]	Loss: 0.931750
Train Epoch: 1 [15360/16080 (95%)]	Loss: 1.119684
Validation Average loss: 1.0630, Accuracy: 1119/1760 (64%)
Train Epoch: 2 [0/16080 (0%)]	Loss: 0.906883
Train Epoch: 2 [1280/16080 (7%)]	Loss: 0.691343
Train Epoch: 2 [2560/16080 (15%)]	Loss: 0.671717
Train Epoch: 2 [3840/16080 (23%)]	Loss: 0.740725
Train Epoch: 2 [5120/16080 (31%)]	Loss: 0.321101
Train 

KeyboardInterrupt: 